# Tutoriel NEOSSat - Traitement et Analytique

**Tutoriel** : Ce tutoriel démontre comment télécharger et charger des fichiers FITS de CADC, effectuer un traitement sommaire d'imagerie, et l'analyse de striures et de mouvement d'objets en Python.<br>

Ce tutoriel utilise des données d'imagerie de la mission NEOSSat.<br>

**Mission et Instrument** : [NEOSSat](https://www.asc-csa.gc.ca/eng/satellites/neossat/) <br>

**Objectifs de la Mission** :
* Connaissance de la situation spatiale : Objets géocroiseurs (astéroïdes, débris spatiaux)
* Cible astronomique : objets célestes (comètes, nébuleuses, galaxies, exoplanètes)

**Exigences du système** : Python 3.9+ (testé)<br>

**Niveau du tutoriel** : Intermédiaire <br>

***
**Licence MIT** <br>
Copyright (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence spatiale canadienne, 2024. <br>
Droit d'auteur (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence Spatiale Canadienne, 2024.<br>

Pour plus d'informations, veuillez vous référer au fichier *License.txt*.

***
**Informations contextuelles** <br>
La mission [NEOSSat](https://www.asc-csa.gc.ca/eng/satellites/neossat/) lancée le 25 février 2013, est le premier télescope spatial au monde dédié à la détection et au suivi d'objets géocroiseurs.

Dans le cadre des efforts du Canada pour observer et suivre les astéroïdes, comètes, satellites et débris spatiaux, ce microsatellite, seulement de la taille d'une valise, est une plateforme d'observation puissante.

Opérant à une altitude d'environ 800 kilomètres, ce microsatellite orbite la Terre toutes les 100 minutes et fournit une surveillance continue des objets géocroiseurs. NEOSSat utilise une technologie avancée de roue de réaction, d'abord démontrée par le satellite [Microvariabilité et Oscillations des Étoiles (MOST)](https://www.asc-csa.gc.ca/eng/satellites/most/) lui permettant de balayer des régions de l'espace près du Soleil, aidant à identifier les astéroïdes et comètes qui peuvent passer près de la Terre.

Dans le cadre de l'engagement du Canada envers la sécurité spatiale, NEOSSat est impliqué dans le suivi des débris spatiaux, fournissant des données essentielles sur les objets qui sont difficiles à observer par les télescopes terrestres. La mission est financée conjointement par l'Agence spatiale canadienne (ASC) et Recherche et développement pour la défense Canada (RDDC). NEOSSat continue à faire progresser le rôle du Canada dans l'exploration spatiale et la sécurité grâce à ses contributions continues au suivi d'astéroïdes, à la surveillance de débris spatiaux, et à la recherche d'exoplanètes.

***
**Lecture complémentaire** <br>
* **Titre :** [Guide de l'utilisateur d'image FITS NEOSSat](https://data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/NEOSSAT/Supporting%20Documents/CSA-NEOSSAT-MAN-0002_FITS_IMAGE_UGUIDE-v4-00.pdf)
    * **Description :** Un document précieux qui contient des informations complètes et détaillées concernant le format FITS de NEOSSat, la sémantique des mots-clés, et les données disponibles dans le fichier FITS.
<p></p>

* **Titre :** [L'Expérience NEOSSat : 5 ans dans la vie du télescope de surveillance spatiale du Canada](https://data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/NEOSSAT/Supporting%20Documents/NEOSSat%20Experience-v1.5.pdf)
    * **Description :** Un papier de résumé utile concernant les développements, défis et succès du programme NEOSSat.
<p></p>

* **Titre :** [Principales découvertes de la mission de microsatellite SSA spatial NEOSSat](https://cradpdf.drdc-rddc.gc.ca/PDFS/unc326/p808033_A1b.pdf)
    * **Description :** Un autre papier de résumé utile concernant les développements, défis et succès du programme NEOSSat.
<p></p>

* **Titre :** [Surveillance spatiale depuis un microsatellite : Traitement d'observation métrique de NEOSSat](https://espace.rmc.ca/jspui/handle/11264/1364)
    * **Description :** Une thèse concernant la surveillance géocroiseur en utilisant NEOSSat, incluant des informations de traitement approfondies et le développement d'algorithmes.

## Bibliothèques Requises
* Ci-dessous, nous initialiserons plusieurs bibliothèques requises pour le tutoriel.

In [ ]:
import os
import requests
import re
import imageio
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.cadc import Cadc
from scipy.ndimage import gaussian_filter, median_filter
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks
from skimage.morphology import (
    binary_closing,
    binary_opening,
    remove_small_objects,
    binary_dilation,
    disk,
)
from skimage.draw import line as draw_line
from pylsd.lsd import lsd
from IPython.display import Image

## Section A - Acquisition d'imagerie NEOSSat
* Information sur le Portail de Données Ouvertes
* Information sur le Centre canadien de données astronomiques
* Recherche, filtrage et téléchargement d'imagerie NEOSSat

Pour acquérir des images NEOSSat, vous pouvez accéder aux données de deux sources principales : le Portail de Données Ouvertes de l'Agence spatiale canadienne (ASC) et le Centre canadien de données astronomiques (CADC).

L'ASC offre les données NEOSSat à travers son [Portail de Données Ouvertes](https://open.canada.ca/data/en/dataset/9ae3e718-8b6d-40b7-8aa4-858f00e84b30), aux côtés d'autres ressources utiles. Ces données peuvent ensuite être téléchargées manuellement ou à travers le Protocole de Transfert de Fichiers (FTP), et sont principalement organisées par date et heure. Pour des informations supplémentaires sur l'accès via FTP, veuillez voir le [tutoriel](https://github.com/asc-csa/NEOSSAT_Tutorial/blob/main/01_%20Extracting%20Data%20and%20Visualization.ipynb) suivant.

Alternativement, le [CADC](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/neossat/) fournit les données NEOSSat avec des capacités de recherche avancées, incluant date-heure, objet cible, paramètres d'acquisition et plus. Ces données peuvent être recherchées en utilisant leur [plateforme](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/search/?collection=NEOSSAT&noexec=true), ou vous pouvez y accéder à travers [astroquery](https://astroquery.readthedocs.io/en/latest/) une bibliothèque Python populaire pour interroger et accéder aux données astronomiques. CADC fournit plusieurs [tutoriels](https://github.com/opencadc/notebook-tutorials/blob/master/astroquery_example_kbos.ipynb) utiles pour aider à mieux comprendre l'intégration d'astroquery avec CADC. Nous nous concentrerons sur l'utilisation d'astroquery pour télécharger et accéder aux données dans ce tutoriel.

D'abord, en utilisant le [Langage de Requête de Données Astronomiques](https://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html) nous développerons une fonction pour créer des requêtes à utiliser dans astroquery. ADQL est un langage similaire au SQL permettant une recherche structurée des bases de données astronomiques.

In [ ]:
def construct_adql_query(start_date, end_date, collection='NEOSSAT', calibration_level='2'):

    start_mjd = Time(start_date).mjd
    end_mjd = Time(end_date).mjd

    # Construct the ADQL query
    adql_query = f"""
    SELECT * FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID
    WHERE (
        INTERSECTS(
            INTERVAL({start_mjd}, {end_mjd}),
            Plane.time_bounds_samples
        ) = 1
        AND Observation.collection = '{collection}'
        AND Plane.calibrationLevel = '{calibration_level}'
        AND (
            Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'
        )
    )
    """
    return adql_query

In [ ]:
def search_neossat_data_with_adql(adql_query):
    # Execute the query using exec_sync
    cadc = Cadc()
    results = cadc.exec_sync(adql_query) 
    return results

Pour fournir une facilité d'accès, nous avons combiné les fonctions ci-dessus en une fonction de recherche par date-heure ci-dessous.

In [ ]:
def search_neossat_data(start_date, end_date):
    # Convert start_date and end_date to MJD (Modified Julian Date)
    start_mjd = Time(start_date).mjd
    end_mjd = Time(end_date).mjd

    # Construct the ADQL query
    adql_query = f"""
    SELECT * FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID
    WHERE (
        INTERSECTS(
            INTERVAL({start_mjd}, {end_mjd}),
            Plane.time_bounds_samples
        ) = 1
        AND Observation.collection = 'NEOSSAT'
        AND Plane.calibrationLevel = '2'
        AND (
            Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'
        )
    )
    """

    # Execute the query using exec_sync
    cadc = Cadc()
    results = cadc.exec_sync(adql_query)

    return results

In [ ]:
# Define the date range for your search in the yyyy-mm-dd format
start_date = '2023-12-05' 
end_date = '2023-12-06'

# Search for NEOSSat data within the date range
results_table = search_neossat_data(start_date, end_date)
print(f"Number of matching observations: {len(results_table)}")

Les résultats incluront des informations telles que les temps d'exposition, les heures de début d'observation, et les propriétés du filtre. Vous pouvez filtrer les données davantage selon vos besoins en utilisant ces champs dans la table retournée. Les filtres typiques peuvent inclure le nom de l'objet, les mots-clés d'instrument, ou le temps d'exposition. Quelques propriétés utiles incluent :

* **productID**
    * Les paramètres incluent cor, cord, clean et autres pour sélectionner le niveau de traitement appliqué (couvert en détail plus tard).
* **instrument_keywords**
    * 06-ST_ACQUIRE : Suivi général des cibles célestes.
    * 16-FINE_POINT : Suivi précis des cibles célestes en mouvement rapide.
* **type**
    * Image d'objet ou sombre.
* **target_name**
    * Nom de la cible céleste suivie.
* **time_exposure**
    * Temps pendant lequel l'obturateur était ouvert et le capteur exposé, ce qui correspond au temps d'imagerie.

In [ ]:
# Optionally, perform additional filtering of results based on additional criteria
# For example, only select observations with exposure time greater than 105 seconds and using fine-point tracking 
results_table = results_table[(results_table['instrument_keywords'] == '16-FINE_POINT') & (results_table['time_exposure'] > 105)]

# Taking only the first five records
filtered_results = results_table[:5]

# Displaying the first result
filtered_results[:1]

### Téléchargement de fichiers FITS NEOSSat

Pour télécharger les données NEOSSat de CADC, nous utiliserons astroquery.cadc.Cadc pour interroger directement CADC.

Le processus de téléchargement peut prendre du temps selon le nombre d'images et les vitesses de réseau. Pour éviter de retélécharger des fichiers, assurez-vous de les garder organisés dans un dossier désigné sur votre ordinateur. Vous pouvez spécifier le nom du dossier pendant le processus de téléchargement comme montré dans le code ci-dessous :

In [ ]:
def download_neossat_data(results_table, folder_name='neossat_data'):
    # Get the current working directory
    current_dir = os.getcwd()
    output_folder = os.path.join(current_dir, folder_name)

    # Create the folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cadc = Cadc()

    # Get a list of data URLs based on the results table
    data_urls = cadc.get_data_urls(results_table)

    filenames = []

    for url in data_urls:
        # Extract the filename from the URL
        filename = os.path.basename(url.split('?')[0])
        filepath = os.path.join(output_folder, filename)
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            filenames.append(filepath)
            print(f"Downloaded {filename}")
        else:
            print(f"Failed to download {url}")

    return filenames

In [ ]:
# Download the data to a local folder
output_folder = 'neossat_example_data'
downloaded_files = download_neossat_data(filtered_results, output_folder)
print(f"Downloaded {len(downloaded_files)} files to {output_folder}")

## Section B - Visualisation et Exploration Initiales
* Exploration initiale des données
* Accès aux images

Lors du travail avec les images FITS NEOSSat, il est important de comprendre la structure des fichiers FITS. Les images NEOSSat utilisent le format [FITS (Système de Transport d'Image Flexible)](https://fits.gsfc.nasa.gov/fits_primer.html), qui contient un en-tête principal et des extensions optionnelles avec des métadonnées comme les données de télémétrie brutes. Chaque image FITS de NEOSSat est typiquement une matrice de pixels 1024x1024, avec chaque pixel correspondant à environ 3 secondes d'arc dans le champ de vision. Les fichiers FITS NEOSSat incluent aussi des métadonnées telles que le temps d'exposition, le filtre, le pointage du télescope, et les paramètres de lecture CCD. Ces fichiers peuvent être ouverts et explorés en utilisant des bibliothèques Python comme astropy ou autres bibliothèques de visualisation d'images.

Les images NEOSSat suivent aussi une convention de nommage stricte basée sur le niveau de traitement et le type d'image. Les noms de fichiers incluent des indicateurs pour les images brutes ou nettoyées, le temps d'acquisition et le type d'instrument.

**Par exemple :**
* NEOS_"detector"_"timestamp".fits
    * Où "detector" est soit "SCI" (pour CCD Science) ou "ST" (pour CCD Star Tracker) et où
    "timestamp" suit la convention "YYYYDDDhhmmss" correspondant au début d'exposition en
    Temps Universel Coordonné (UTC).
    * Les images nettoyées auront le suffixe _cor.fits pour les images nettoyées Python, et _clean.fits pour les images nettoyées Java.
    * Les images nettoyées avec soustraction d'image sombre auront le suffixe _cord.fits.

Plus de détails sur les conventions de nommage et la liste complète d'en-tête sont disponibles dans le [Guide de l'utilisateur d'image FITS](https://data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/NEOSSAT/Supporting%20Documents/CSA-NEOSSAT-MAN-0002_FITS_IMAGE_UGUIDE-v4-00.pdf).

In [ ]:
def read_fits_data(filename):
    data = fits.getdata(filename)
    data = data.astype(np.float64)
    return data

In [ ]:
fits_file = 'neossat_example_data/NEOS_SCI_2023339134953_cor.fits'  # Replace with the path to your FITS file
example_image = read_fits_data(fits_file)

In [ ]:
plt.figure(figsize=(10, 10))
vmin, vmax = np.percentile(example_image, [5, 95])
plt.imshow(example_image, cmap='gray', vmin=vmin, vmax=vmax)
plt.title('Example Image')
plt.colorbar()
plt.show()

In [ ]:
# Access the fits file header 
hdul = fits.open(fits_file)

# Print the header of the primary HDU (Header Data Unit)
print(hdul[0].header[:5])

# Close the FITS file after processing
hdul.close()

## Section C - Prétraitement d'Images
* Application de la correction d'overscan grossière
* Processus pour développer les images sombres

Avant d'analyser les images FITS NEOSSat, vous pourriez avoir besoin d'effectuer plusieurs étapes de prétraitement si les images ne contiennent pas de suffixe _cor, _cord ou _clean.

Ces étapes et d'autres aident à s'assurer que les données d'image sont propres et libres d'artefacts et de [pixels chauds](https://www.astropy.org/ccd-reduction-and-photometry-guide/v/dev/notebooks/08-01-Identifying-hot-pixels.html). Une étape commune est la correction d'overscan, qui supprime les valeurs de pixels supplémentaires en dehors de la zone CCD qui ne correspondent pas aux données d'image réelles. De plus, la soustraction de trame sombre et la correction de champ plat sont typiquement nécessaires pour tenir compte du bruit et des variations de sensibilité des pixels à travers le détecteur.

Pour un prétraitement avancé et une analyse d'image, il est hautement recommandé d'explorer la [Bibliothèque NEOSSat](https://github.com/jasonfrowe/neossat/tree/master) développée par Jason Rowe. L'utilisation d'exemple de la bibliothèque est disponible [ici](https://github.com/jasonfrowe/neossat/blob/master/notebooks/Cleaning%20NEOSSat%20Photometry.ipynb).

In [ ]:
def get_image_dimensions(filename, overscan_width=50):
    header = fits.getheader(filename)
    x_size = header['NAXIS1']
    y_size = header['NAXIS2']
    overscan_start_col = x_size - overscan_width
    return x_size, y_size, overscan_start_col

In [ ]:
def overscan_correction(science_data, overscan_region):
    overscan_median = np.median(overscan_region, axis=1)
    corrected_data = science_data - overscan_median[:, np.newaxis]
    return corrected_data

In [ ]:
def create_master_dark(dark_frames_directory):
    dark_files = [f for f in os.listdir(dark_frames_directory) if f.endswith('.fits')]
    dark_data_list = []

    for filename in tqdm(dark_files, desc='Processing dark frames'):
        input_path = os.path.join(dark_frames_directory, filename)
        data = read_fits_data(input_path)
        x_size, y_size, overscan_start_col = get_image_dimensions(input_path)
        science_data = data[:, :overscan_start_col]
        overscan_region = data[:, overscan_start_col:]
        corrected_dark = overscan_correction(science_data, overscan_region)
        dark_data_list.append(corrected_dark)

    # Stack dark frames using median to reduce noise and reject outliers
    dark_stack = np.array(dark_data_list)
    master_dark = np.median(dark_stack, axis=0)
    return master_dark

In [ ]:
def apply_dark_correction(overscan_corrected_directory, master_dark, output_directory):
    cor_files = [f for f in os.listdir(overscan_corrected_directory) if f.endswith('_cor.fits')]

    for filename in tqdm(cor_files, desc='Applying dark correction'):
        input_path = os.path.join(overscan_corrected_directory, filename)

        # Modify the output filename to include '_cord'
        base_filename = os.path.splitext(filename)[0]
        new_filename = base_filename.replace('_cor', '_cord') + '.fits'
        output_path = os.path.join(output_directory, new_filename)

        # Read the overscan-corrected data
        data = read_fits_data(input_path)

        # Ensure data and master_dark have the same shape
        if data.shape != master_dark.shape:
            print(f"Warning: Shape mismatch between data {data.shape} and master dark {master_dark.shape}")
            # Crop or pad as necessary (here we crop to the smallest shape)
            min_rows = min(data.shape[0], master_dark.shape[0])
            min_cols = min(data.shape[1], master_dark.shape[1])
            data = data[:min_rows, :min_cols]
            master_dark_cropped = master_dark[:min_rows, :min_cols]
        else:
            master_dark_cropped = master_dark

        # Subtract master dark
        dark_corrected_data = data - master_dark_cropped

        # Save the dark-corrected image
        header = fits.getheader(input_path)
        header.add_history('Dark correction applied.')
        hdu = fits.PrimaryHDU(dark_corrected_data, header=header)
        hdu.writeto(output_path, overwrite=True)

In [ ]:
# Define directories; they will use your current working directory
input_directory = 'neossat_example_data' # Replace with your raw images directory
output_directory = 'neossat_example_results'  # Replace with your processed images directory
dark_frames_directory = 'dark_test'     # Replace with your dark frames directory

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
# List FITS files
fits_files = [f for f in os.listdir(input_directory) if f.endswith('.fits')]

# Overscan correction
print("Starting overscan correction...")
for filename in tqdm(fits_files, desc='Processing images'):
    input_path = os.path.join(input_directory, filename)

    # Modify the output filename to include '_cor' before the extension
    base_filename = os.path.splitext(filename)[0]
    new_filename = base_filename + '_cor.fits'
    output_path = os.path.join(output_directory, new_filename)

    # Read the image data
    data = read_fits_data(input_path)
    x_size, y_size, overscan_start_col = get_image_dimensions(input_path)
    science_data = data[:, :overscan_start_col]
    overscan_region = data[:, overscan_start_col:]
    corrected_data = overscan_correction(science_data, overscan_region)

    # Save the corrected image
    header = fits.getheader(input_path)
    header.add_history('Overscan correction applied.')
    hdu = fits.PrimaryHDU(corrected_data, header=header)
    hdu.writeto(output_path, overwrite=True)
print("Overscan correction completed.")

In [ ]:
# Create master dark frame
print("Creating master dark frame...")
master_dark = create_master_dark(dark_frames_directory)
master_dark_path = os.path.join(output_directory, 'master_dark.fits')
hdu = fits.PrimaryHDU(master_dark)
hdu.writeto(master_dark_path, overwrite=True)
print("Master dark frame created.")

In [ ]:
# Apply dark correction to overscan-corrected images
print("Applying dark correction...")
apply_dark_correction(output_directory, master_dark, output_directory)
print("Dark correction completed.")

In [ ]:
# Specify image type ('_cor' for overscan-corrected, '_cord' for dark-corrected, or '_clean' for alternative overscan-corrected)
image_type = '_cord'

# List images of the specified type
image_files = [f for f in os.listdir(output_directory) if f.endswith(f'{image_type}.fits')]
image_files.sort()  # Sort the list for consistency

print(f"Found {len(image_files)} images of type {image_type}")

Ensuite, nous appliquerons l'empilement d'images, une technique commune en traitement d'image pour réduire le bruit de fond et améliorer la qualité globale d'image. L'empilement de plusieurs images brutes résulte en une image finale avec une clarté significativement meilleure que n'importe quelle image brute unique.

In [ ]:
def stack_images(image_files, image_directory):
    image_data_list = []
    for filename in tqdm(image_files, desc='Reading images'):
        filepath = os.path.join(image_directory, filename)
        data = read_fits_data(filepath)
        image_data_list.append(data)
    
    image_shape = image_data_list[0].shape
    stacked_image = np.median(np.array(image_data_list), axis=0)
    return stacked_image, image_shape

In [ ]:
# Stack the images
print('Stacking images...')
stacked_image, image_shape = stack_images(image_files, output_directory)

In [ ]:
# Save the stacked image
stacked_image_path = os.path.join(output_directory, f'stacked_image{image_type}.fits')
hdu = fits.PrimaryHDU(stacked_image)
hdu.writeto(stacked_image_path, overwrite=True)
print(f'Stacked image saved to {stacked_image_path}')

## Section D - Analyse d'Image et Détection
* Détection de segments de ligne pour détecter les striures
* Détection de mouvement pour détecter les objets géocroiseurs

Cette section vous guidera à travers le processus d'analyse d'imagerie NEOSSat, se concentrant sur la détection de striures de mouvement dans les trames d'image. Les images haute résolution de NEOSSat sont idéales pour détecter des objets faibles, et le mode de suivi d'étoile fournit des informations supplémentaires sur le mouvement d'objet relatif au champ d'étoile. La détection d'objets en mouvement peut être particulièrement importante pour identifier des objets géocroiseurs (NEO) ou des débris spatiaux.

Une des premières étapes pour détecter des objets dans des images astronomiques est d'appliquer la détection de contour pour mettre en évidence les objets potentiels d'intérêt. Le [LSD : Détecteur de Segment de Ligne](http://www.ipol.im/pub/art/2012/gjmr-lsd/) est particulièrement utile dans ce cas, car il peut efficacement identifier les limites même dans des images bruyantes comme celles capturées dans des environnements spatiaux. La documentation supplémentaire sur l'implémentation de cet algorithme en Python est disponible [ici](https://github.com/primetang/pylsd?tab=readme-ov-file).

In [ ]:
def detect_streaklets(image_data):
    # Preprocess the image
    smoothed = gaussian_filter(image_data, sigma=1)
    normalized = (smoothed - np.min(smoothed)) / (np.max(smoothed) - np.min(smoothed))
    img_uint8 = (normalized * 255).astype(np.uint8)

    # Apply LSD
    lines = lsd(img_uint8)

    # Create an empty mask
    streaklet_mask = np.zeros_like(img_uint8, dtype=bool)

    # Define minimum length for line segments
    min_length = 10  # Adjust as needed

    for line in lines:
        x0, y0, x1, y1, width = line
        length = np.hypot(x1 - x0, y1 - y0)
        if length >= min_length:
            rr, cc = draw_line(int(y0), int(x0), int(y1), int(x1))
            rr = np.clip(rr, 0, img_uint8.shape[0] - 1)
            cc = np.clip(cc, 0, img_uint8.shape[1] - 1)
            streaklet_mask[rr, cc] = True

    # Morphological operations to clean up the mask
    streaklet_mask = binary_dilation(streaklet_mask, footprint=disk(1))
    streaklet_mask = remove_small_objects(streaklet_mask, min_size=50)

    return streaklet_mask

Les images NEOSSat sont souvent prises en séquences qui permettent la détection d'objets en mouvement en comparant les changements à travers les trames. Vous pouvez détecter des objets en mouvement en calculant la différence entre des trames consécutives pour mettre en évidence les changements qui pourraient indiquer le mouvement d'objet.

Voici un exemple de comment calculer la différence entre deux trames pour détecter le mouvement :

In [ ]:
def detect_moving_objects(image_files, image_directory, image_shape):
    num_images = len(image_files)
    data_cube = np.zeros((num_images, *image_shape))

    for idx, filename in enumerate(tqdm(image_files, desc='Loading images for moving object detection')):
        filepath = os.path.join(image_directory, filename)
        data_cube[idx] = read_fits_data(filepath)

    # Calculate the median image to use as a reference (background)
    median_image = np.median(data_cube, axis=0)

    # Subtract the median image from each frame to detect changes
    difference_cube = data_cube - median_image

    # Apply threshold to detect significant changes
    threshold = np.std(difference_cube) * 3  # Adjust the factor as needed
    moving_objects_mask = np.max(difference_cube > threshold, axis=0)

    # Clean up the mask
    moving_objects_mask = binary_opening(moving_objects_mask, footprint=np.ones((3, 3)))
    moving_objects_mask = remove_small_objects(moving_objects_mask, min_size=20)

    return moving_objects_mask

In [ ]:
# Detect streaklets in individual images
print('Detecting streaklets in individual images...')
streaklet_masks = []
for filename in tqdm(image_files, desc='Detecting streaklets'):
    filepath = os.path.join(output_directory, filename)
    data = read_fits_data(filepath)
    streaklet_mask = detect_streaklets(data)
    streaklet_masks.append(streaklet_mask)
    # Save the streaklet mask
    mask_filename = os.path.splitext(filename)[0] + '_streaklet_mask.fits'
    mask_path = os.path.join(output_directory, mask_filename)
    hdu = fits.PrimaryHDU(streaklet_mask.astype(np.uint8))
    hdu.writeto(mask_path, overwrite=True)
print('Streaklet detection completed.')

In [ ]:
# Detect moving objects across images
print('Detecting moving objects across images...')
moving_objects_mask = detect_moving_objects(image_files, output_directory, image_shape)
# Save the moving objects mask
moving_objects_mask_path = os.path.join(output_directory, f'moving_objects_mask{image_type}.fits')
hdu = fits.PrimaryHDU(moving_objects_mask.astype(np.uint8))
hdu.writeto(moving_objects_mask_path, overwrite=True)
print(f'Moving objects mask saved to {moving_objects_mask_path}')

Cette technique peut être davantage améliorée en appliquant des filtres ou un seuillage pour réduire le bruit et mettre en évidence seulement les changements significatifs, aidant à isoler les objets en mouvement tels que les satellites ou astéroïdes.

Une fois le mouvement détecté, la prochaine étape est de suivre l'objet à travers plusieurs trames. Les algorithmes de suivi d'objet, tels que le suivi de centroïde, peuvent être employés pour suivre le mouvement de l'objet détecté au fil du temps. En analysant la trajectoire et la vitesse, nous pouvons dériver des informations significatives sur le mouvement de l'objet.

## Section E - Visualisation
* Tracé de masques d'image
* Création de visualisation GIF

Dans cette section, nous visualisons les résultats de nos étapes de traitement. Vous verrez l'image empilée finale, ainsi que le masque de mouvement qui met en évidence le mouvement détecté à travers les trames. De plus, cette section fournit l'option de créer un GIF animé de l'objet en mouvement, vous permettant d'observer son mouvement au fil du temps.

In [ ]:
def visualize_results(stacked_image, streaklet_masks, moving_objects_mask):
    # Display the stacked image
    plt.figure(figsize=(10, 10))
    vmin, vmax = np.percentile(stacked_image, [5, 95])
    plt.imshow(stacked_image, cmap='gray', vmin=vmin, vmax=vmax)
    plt.title('Stacked Image')
    plt.colorbar()
    plt.show()

    # Display the streaklets on a sample image
    sample_idx = len(streaklet_masks) // 2
    sample_mask = streaklet_masks[sample_idx]
    plt.figure(figsize=(10, 10))
    plt.imshow(sample_mask, cmap='gray')
    plt.title('Streaklet Mask (Sample Image)')
    plt.show()

    # Display the moving objects mask
    plt.figure(figsize=(10, 10))
    plt.imshow(moving_objects_mask, cmap='gray')
    plt.title('Moving Objects Mask')
    plt.show()

In [ ]:
# Visualize the results
visualize_results(stacked_image, streaklet_masks, moving_objects_mask)

In [ ]:
def create_gif(image_files, image_directory, output_path, duration=0.5):
    frames = []
    for filename in tqdm(image_files, desc='Creating GIF'):
        filepath = os.path.join(image_directory, filename)
        data = read_fits_data(filepath)

        # Normalize the image data for visualization
        vmin, vmax = np.percentile(data, [5, 95])
        normalized_data = np.clip((data - vmin) / (vmax - vmin), 0, 1)

        # Convert to 8-bit grayscale image
        image_8bit = (normalized_data * 255).astype(np.uint8)

        # Append the image to the frames list
        frames.append(image_8bit)

    # Save the frames as an animated GIF and set loop to 0 for infinite looping
    imageio.mimsave(output_path, frames, format='GIF', duration=duration, loop=0)
    print(f"Animated GIF saved to {output_path}")

In [ ]:
# Define the output path for the GIF
gif_output_path = os.path.join(output_directory, 'animated_images.gif')

# Create the GIF
create_gif(image_files, output_directory, gif_output_path, duration=0.5)

In [ ]:
# Display the GIF
Image(filename=gif_output_path)

## Interprétation des Résultats

À partir des images ci-dessus, nous pouvons voir qu'beaucoup de mouvement a été détecté entre les trames d'image. Ceci est en partie parce qu'aucune correction de dérive n'a été appliquée entre les piles d'images résultant en un décalage horizontal des objets. Cependant, pour notre objet d'intérêt, nous pouvons voir un mouvement diagonal entre les points. Le traitement du ["flux optique"](https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html) et la direction des vecteurs peut être une technique pour déterminer les objets d'intérêt.

De plus, il est à noter que dans d'autres séries d'images, vous pourriez rencontrer de plus grandes striures. Celles-ci seront de grandes lignes à travers l'image et sont typiquement seulement visibles dans une seule trame. Elles représentent des objets à haute vitesse passant devant le capteur tels que des débris spatiaux ou des satellites proches.

## Félicitations !

Vous avez maintenant traité les données NEOSSat. C'est un ensemble de données complexe, avec de nombreuses façons de traitement. Donc, nous vous recommandons de continuer à explorer cet ensemble de données et de trouver de nouvelles façons excitantes de détecter et suivre les objets géocroiseurs.

L'ensemble de données NEOSSat peut aussi être utilisé pour créer d'autres produits de données précieux, tels que des diagrammes de courbe de lumière. Ces diagrammes sont cruciaux pour la détection et l'analyse d'exoplanètes, car ils suivent les variations de la luminosité d'une étoile au fil du temps, aidant à identifier les transits d'exoplanètes potentiels.